In [147]:
import json
import pandas as pd
import requests

In [148]:
follows = pd.read_csv('../shared-folder-gald/data/follow-link.csv')

In [149]:
usernames = pd.unique(follows[['source', 'target']].values.ravel()).tolist()

In [150]:
with open("../shared-folder-gald/keys3.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [151]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [152]:
# def get_access_token(client_key, client_secret):
#     endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {
#         'client_key': client_key,
#         'client_secret': client_secret,
#         'grant_type': 'client_credentials',
#     }

#     response = requests.post(endpoint_url, headers=headers, data=data)
#     response_json = response.json()

#     if response.status_code == 200:
#         print(response_json)  # Debugging print
#         return response_json.get('access_token', 'No access token found')
#     else:
#         print(f"Error {response.status_code}: {response_json}")  # Debugging print
#         return response_json

In [153]:
def get_liked_info(username, access_token,fields ="id, username" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/liked_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [154]:
access_token = get_access_token(client_key, client_secret)

KeyError: 'access_token'

In [ ]:
liked_videos = []
for username in usernames[10989:11990]:
    liked_videos.append(get_liked_info(username, access_token))

In [ ]:
combined = {}
for i, username in enumerate(usernames[10989:11990]):
    combined[username] = liked_videos[i]

In [ ]:
rows = []

for key, value in combined.items():
    if value and 'user_liked_videos' in value:  # Ensure key has data
        for video in value['user_liked_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'id': video['id'],  # Video ID
                'creator': video['username']  # Video username
            })

# Create DataFrame
likes_df = pd.DataFrame(rows)

# Display DataFrame
print(likes_df)

                 username                   id               creator
0             chamokitepr  7473628913940385029       ibarrechejavier
1             chamokitepr  7462420858146000134           mafecardozo
2             chamokitepr  7465439879036488966             showderrr
3             chamokitepr  7470690335392419077         emirabdulgani
4             chamokitepr  7453512901916216618         andrescooking
...                   ...                  ...                   ...
1558  drasifiqbalkhan1121  7473126937997659414      umargulofficiall
1559  drasifiqbalkhan1121  7474566951885393160  rutbarainbowofficial
1560  drasifiqbalkhan1121  7473562100191759623  psysalmankhangigyani
1561  drasifiqbalkhan1121  7474029746712595720         ali.writes.z1
1562  drasifiqbalkhan1121  7466464922092834065          you_lost_him

[1563 rows x 3 columns]


In [ ]:
likes_df.to_csv("liked_videos.csv", mode="a", index=False, header=False)